In [1]:
import hyperspy.api as hp


Logging is active
The log is stored in the hyperspy_log.py file in the current directory


In [2]:
%matplotlib qt5

In [3]:
s = hp.load('/Users/veersaysit/Desktop/EELS data/GaAs100_Q4_EELStest_130705/Pos1_20muCA/Profile Of Pos 1 EELS_1s.dm3')

In [4]:
s

<EELSSpectrum, title: Profile Of Pos 1 EELS_1s, dimensions: (|1024)>

In [5]:
s.plot()

In [6]:
s.align_zero_loss_peak()


Initial ZLP position statistics
-------------------------------
Summary statistics
------------------
mean:	0.000
std:	0.000

min:	0.000
Q1:	0.000
median:	0.000
Q3:	0.000
max:	0.000


In [7]:
s.set_signal_type("EELS")

In [8]:
s.metadata

├── General
│   ├── original_filename = Profile Of Pos 1 EELS_1s.dm3
│   └── title = Profile Of Pos 1 EELS_1s
└── Signal
    ├── Noise_properties
    │   └── Variance_linear_model
    │       ├── gain_factor = 0.8199999928474426
    │       └── gain_offset = 0.0
    ├── binned = True
    ├── quantity = Electrons (Counts)
    └── signal_type = EELS

In [9]:
s.set_microscope_parameters(beam_energy=197, convergence_angle=10, collection_angle=20)

In [10]:
m = s.create_model(auto_background=False)

In [11]:
m.components

   # |      Attribute Name |      Component Name |      Component Type
---- | ------------------- | ------------------- | -------------------

In [12]:
Lg = hp.model.components1D.Lorentzian()

In [13]:
m.append(Lg)

In [14]:
m.components

   # |      Attribute Name |      Component Name |      Component Type
---- | ------------------- | ------------------- | -------------------
   0 |          Lorentzian |          Lorentzian |          Lorentzian

In [15]:
m.fit_component(Lg)

In [17]:
m.plot()

In [18]:
Lg

<Lorentzian (Lorentzian component)>

In [19]:
Lg.A.as_signal

<bound method Parameter.as_signal of <Parameter A of Lorentzian component>>

In [20]:
m.components

   # |      Attribute Name |      Component Name |      Component Type
---- | ------------------- | ------------------- | -------------------
   0 |          Lorentzian |          Lorentzian |          Lorentzian

In [21]:
PL = m.as_signal()

In [22]:
PL.plot()

In [23]:
sgp = s-PL

In [24]:
sgp.plot()

In [33]:
import scipy.io as sio

In [34]:
mat_contents = sio.loadmat('Sg_combined.mat')
#Note that while saving .mat file in matlab always make sure it is in (1,1024) dimention and not in (1024,1).

In [35]:
mat_contents

{'Sg1p': array([[  0.35808201,   0.48607588,   0.21403801, ...,  12.09422819,
          13.56541954,  13.70325256]]),
 '__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Tue Dec  5 12:41:35 2017',
 '__version__': '1.0',
 'eSg1p': array([[  3.58082014e-01,   4.86075884e-01,   2.14038008e-01, ...,
           7.99405044e-06,   7.83235436e-06,   7.67392893e-06]])}

In [37]:
Sg1p = mat_contents['Sg1p']
Sg1p.shape

(1, 1024)

In [38]:
Sg1p = hp.signals.Signal1D(Sg1p)

In [39]:
Sg1p.set_signal_type('EELS')

In [40]:
Sg1p.metadata

├── General
│   └── title = 
└── Signal
    ├── binned = True
    └── signal_type = EELS

In [41]:
Sg1p.plot()

In [42]:
eSg1p = mat_contents['eSg1p']
eSg1p = hp.signals.Signal1D(eSg1p)
eSg1p.set_signal_type('EELS')

In [49]:
flSg1p = Sg1p.fourier_log_deconvolution(zlp=eSg1p)

In [44]:
flSg1p.plot()

In [46]:
sio.savemat("flSg1p",{"flSg1p":flSg1p.data})

In [47]:
flSg1p.data

array([[-0.3304818 , -0.45083244, -0.17165334, ...,  0.02785517,
        -0.27016352, -0.41231141]])